In [1]:
import matplotlib
import torch
import torchvision.models as models
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchvision.transforms import transforms
from torch.autograd import Variable
import numpy as np
import torch.optim as optim

import matplotlib.pyplot as plt

from tools.Tester import ModelNetTester
from tools.ImgDataset import MultiviewImgDataset, SingleImgDataset
from models.MVCNN import MVCNN, SVCNN

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

model1 = SVCNN("cnet1").to(device)
model1.load_state_dict(torch.load('E:\\Work\\MachineLearning\\results\\mvcnn_stage_1\\mvcnn\\model-00004.pth'))

model2 = MVCNN("cnet2", model1).to(device)
model2.load_state_dict(torch.load('E:\\Work\\MachineLearning\\results\\mvcnn_stage_2\\mvcnn\\model-00002.pth'))

val_path = "modelnet40_images_new_12x/*/test"
val_dataset = SingleImgDataset(val_path, scale_aug=False, rot_aug=False, test_mode=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=0)
print('num_val_files: '+str(len(val_dataset.filepaths)))

c:\Users\Denjanzi\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Denjanzi\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG11_Weights.IMAGENET1K_V1`. You can also use `weights=VGG11_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


num_val_files: 29616


In [2]:
model1.eval()
model2.eval()

model = model2
modelName = 'mvcnn'



val_dataset = MultiviewImgDataset(val_path, scale_aug=False, rot_aug=False, num_views=12)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=False, num_workers=0)

trainer = ModelNetTester(model, val_loader, nn.CrossEntropyLoss(), modelName,
                          num_views=1)
trainer.test_embedding(0, 1)
#trainer.test_accuracy()

Predicted Class Names: ['dresser']
Test: 0, Correct Points: 1/1
Predicted Class Names: ['monitor']
Test: 1, Correct Points: 1/1
Predicted Class Names: ['stool']
Test: 2, Correct Points: 1/1
Predicted Class Names: ['bed']
Test: 3, Correct Points: 1/1
Predicted Class Names: ['tv_stand']
Test: 4, Correct Points: 0/1
Predicted Class Names: ['plant']
Test: 5, Correct Points: 1/1
Predicted Class Names: ['table']
Test: 6, Correct Points: 1/1
Predicted Class Names: ['bottle']
Test: 7, Correct Points: 1/1
Predicted Class Names: ['plant']
Test: 8, Correct Points: 1/1
Predicted Class Names: ['table']
Test: 9, Correct Points: 0/1
Predicted Class Names: ['plant']
Test: 10, Correct Points: 1/1
Predicted Class Names: ['monitor']
Test: 11, Correct Points: 1/1
Predicted Class Names: ['range_hood']
Test: 12, Correct Points: 1/1
Predicted Class Names: ['dresser']
Test: 13, Correct Points: 1/1
Predicted Class Names: ['range_hood']
Test: 14, Correct Points: 1/1
Predicted Class Names: ['cup']
Test: 15, Corr